In [17]:
pip install nltk

In [18]:
pip show streamlit

Name: streamlitNote: you may need to restart the kernel to use updated packages.

Version: 1.36.0
Summary: A faster way to build and share data apps
Home-page: https://streamlit.io
Author: Snowflake Inc
Author-email: hello@streamlit.io
License: Apache License 2.0
Location: C:\Users\DELL\anaconda3\Lib\site-packages
Requires: altair, blinker, cachetools, click, gitpython, numpy, packaging, pandas, pillow, protobuf, pyarrow, pydeck, requests, rich, tenacity, toml, tornado, typing-extensions, watchdog
Required-by: 


In [19]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary datasets
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Get English stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Tokenize the text
    words = word_tokenize(text)

    # Lemmatize and remove stopwords
    processed_words = [lemmatizer.lemmatize(word.lower()) for word in words if word.lower() not in stop_words and word.isalpha()]

    # Join the processed words back into a string
    return ' '.join(processed_words)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
import pandas as pd
# df = pd.read_excel('/content/youtube_data.xlsx')
df = pd.read_excel("youtube_data.xlsx")

tages_list = list(df['tags_list'])
title_list = list(df['title_list'])
video_url_list = list(df['video_url_list'])
# video_url_list_cleaned = []
# for i in video_url_list:
#   video_url_list_cleaned.append('https://www.youtube.com/watch?v=' + i.split('?')[1])
# df['video_url_list'] = video_url_list_cleaned

def tags_cleaning(x):
  x = x.replace('[', '')
  x = x.replace(']', '')
  x = x.replace("'", '')
  x = x.split(', ')
  return x

tags_list_cleaned = []
for i in tages_list:
  tags_list_cleaned.append(tags_cleaning(i))

df['tags_list'] = tags_list_cleaned

tags_list = list(df["tags_list"])

df.to_excel('youtub_data_labelled_cleaned.xlsx', index = False)
df.to_json('youtub_data_labelled_cleaned.json', orient='records', lines=True)
video_url_list = list(df['video_url_list'])

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Your data
data = tags_list

# Combine the tags into a single string for each list
documents = [" ".join(tags) for tags in data]

documents = [preprocess_text(text) for text in documents]

# Vectorization
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(documents).toarray()

# Perform hierarchical clustering
Z = linkage(X, method='ward')

# Choose the number of clusters
k = 250
labels = fcluster(Z, k, criterion='maxclust')

In [22]:
# Calculate the centroid of each cluster
cluster_centroids = []
for cluster_id in np.unique(labels):
    cluster_vectors = X[labels == cluster_id]
    cluster_centroid = cluster_vectors.mean(axis=0)
    cluster_centroids.append(cluster_centroid)

In [23]:
import pickle

with open('clustering_labels.pkl', 'wb') as f:
    pickle.dump(labels, f)

# # Save the TF-IDF vectorized matrix
# with open('tfidf_matrix.pkl', 'wb') as f:
#     pickle.dump(X, f)

# Save the vectorizer itself
with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

with open('cluster_centroids.pkl', 'wb') as f:
    pickle.dump(cluster_centroids, f)

In [24]:
# Load the clustering labels
with open('clustering_labels.pkl', 'rb') as f:
    labels = pickle.load(f)

# # Load the TF-IDF vectorized matrix
# with open('tfidf_matrix.pkl', 'rb') as f:
#     X = pickle.load(f)

# Load the vectorizer
with open('vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

with open('cluster_centroids.pkl', 'rb') as f:
    cluster_centroids = pickle.load(f)

In [25]:
for i in range(12):
    z = X[i*400:i*400+400]
    # Save the TF-IDF vectorized matrix
    with open(f'tfidf_matrix_{i}.pkl', 'wb') as f:
        pickle.dump(z, f)
    print(f'tfidf_matrix_{i}.pkl')



tfidf_matrix_0.pkl
tfidf_matrix_1.pkl
tfidf_matrix_2.pkl
tfidf_matrix_3.pkl
tfidf_matrix_4.pkl
tfidf_matrix_5.pkl
tfidf_matrix_6.pkl
tfidf_matrix_7.pkl
tfidf_matrix_8.pkl
tfidf_matrix_9.pkl
tfidf_matrix_10.pkl
tfidf_matrix_11.pkl


In [26]:

z = []
for i in range(12):
    try:
        # Load the TF-IDF vectorized matrix
        with open(f'tfidf_matrix_{i}.pkl', 'rb') as f:
            z.append(pickle.load(f))
            
        print(f'read tfidf_matrix_{i}.pkl')
    except:
        print(f'read tfidf_matrix_{i}.pkl not found')
X = np.vstack(z)  

read tfidf_matrix_0.pkl
read tfidf_matrix_1.pkl
read tfidf_matrix_2.pkl
read tfidf_matrix_3.pkl
read tfidf_matrix_4.pkl
read tfidf_matrix_5.pkl
read tfidf_matrix_6.pkl
read tfidf_matrix_7.pkl
read tfidf_matrix_8.pkl
read tfidf_matrix_9.pkl
read tfidf_matrix_10.pkl
read tfidf_matrix_11.pkl


In [27]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def recommend_videos_using_clustering(keyword, vectorizer, X, labels, df, cluster_centroids, top_n=5):
    # Preprocess the keyword
    keyword_vector = vectorizer.transform([keyword]).toarray()

    # Find the most similar cluster to the keyword
    cluster_similarities = cosine_similarity(keyword_vector, cluster_centroids)
    most_similar_cluster_id = np.argmax(cluster_similarities)

    # Get all videos in the most similar cluster
    cluster_indices = np.where(labels == most_similar_cluster_id + 1)[0]  # +1 because labels are 1-indexed
    cluster_vectors = X[cluster_indices]

    # Calculate similarity of the keyword to all videos in the cluster
    video_similarities = cosine_similarity(keyword_vector, cluster_vectors)[0]

    # Get the indices of the top N matches within the cluster
    top_indices = np.argsort(video_similarities)[::-1][:top_n]

    # Retrieve the corresponding titles and video URLs
    recommendations = df.iloc[cluster_indices[top_indices]][['title_list', 'video_url_list']]

    return recommendations

# Example usage
keyword =input()
recommendations = recommend_videos_using_clustering(keyword, vectorizer, X, labels, df, cluster_centroids)
result_title_list = list(recommendations['title_list'])
result_video_url_list = list(recommendations['video_url_list'])
for i, j in zip(result_title_list, result_video_url_list):
  print(i)
  print(j)

samsung
Samsung Galaxy S10, S10 Plus & S10e ⚡⚡⚡ Everything You Need to Know!
https://www.youtube.com/watch?v=xy-I4PAZqZY
Samsung Galaxy S8 & S8+ Review The Best Android Smartphone?
https://www.youtube.com/watch?v=GVZWnLbvQDE
Samsung Galaxy M Series (M10, M20, M30) Smartphones - Will They Challenge Xiaomi's Dominance?
https://www.youtube.com/watch?v=nnrl8kBooGo
Samsung Galaxy C9 Pro Unboxing & Overview (Indian Unit)
https://www.youtube.com/watch?v=V7eIT3m5bOc
Samsung Galaxy S7 Unboxing & Overview Indian Retail Unit
https://www.youtube.com/watch?v=l-jTHyN810g
